# Creación de archivos .csv para poblar la base de datos

In [1]:
import pandas as pd
import numpy as np

import data_const as c
import utils as u
import relations as r


## Entidad Cliente

In [2]:
try:
    cliente = pd.read_csv(c.filepath_client)

except:
    print('Creando entidad Cliente')
    n_client = c.cardinality['client']
    
    data_names_masc = pd.read_csv(c.path_masc_names, encoding='latin-1')
    data_names_fem = pd.read_csv(c.path_fem_names, encoding='latin-1')
    data_surnames = pd.read_csv(c.path_surnames, encoding='latin-1')

    names_list = u.get_names(data_names_masc, 
                             data_names_fem, 
                             data_surnames,
                            n_client,
                            1)
    # Names
    names = names_list[0]
    surnames_1 = names_list[1]
    surnames_2 = names_list[2]
    names_masc = names_list[3]
    names_fem = names_list[4]
    n_client, n_masc, n_fem = names_list[5]
    
    # Curps
    curp_masc = list(u.get_curps(names_masc,
                             surnames_1[0: n_masc],
                             surnames_2[0: n_masc],
                             sex='M'))
    curp_fem = list(u.get_curps(names_fem,
                            surnames_1[n_masc:],
                            surnames_2[n_masc:],
                            sex='F'))
    curps = curp_masc + curp_fem

    # Direcciones
    cdmx_data = pd.read_csv(c.filepath_raw + '/all_cdmx_data.csv')
    np.random.seed(1)
    to_select = np.random.choice(np.arange(0, cdmx_data.index[-1]), size=n_client, replace=True)
    cdmx_client = cdmx_data.iloc[to_select]
    cdmx_client.astype({'cp': 'float32'})
    
    unities = u.get_unity(curps)
    
    fac, ins, uni = u.get_fac_ins_uni(unities[0], unities[1], unities[2])
    
    # Email
    emails = u.get_mails(names,
                        surnames_1,
                        surnames_2,
                        curps,
                        fac,
                        ins,
                        uni)
    
    # Creamos y rellenamos
    cliente = pd.DataFrame(r.dict_to_df_cliente)

    cliente['curp'] = np.array(curps, dtype=str)
    cliente['nombre'] = np.array(names, dtype=str)
    cliente['a_paterno'] = np.array(surnames_1, dtype=str)
    cliente['a_materno'] = np.array(surnames_2, dtype=str)
    cliente['calle_num'] = np.array(cdmx_client['calle_num'], dtype=str)
    cliente['municipio'] = np.array(cdmx_client['alcaldia'], dtype=str)
    cliente['cp'] = np.array(cdmx_client['cp'], dtype=int)
    cliente['colonia'] = np.array(cdmx_client['colonia'], dtype=str)
    cliente['estado'] = str(c.address)
    cliente['tel_casa'] = np.array(u.get_tels(n_client), dtype=int)
    cliente['tel_celular'] = np.array(u.get_tels(n_client), dtype=int)
    cliente['fotografia'] = np.array(u.get_photos(curps), dtype=str)
    cliente['es_alumno'] = np.array(unities[0], dtype=bool)
    cliente['es_academico'] = np.array(unities[1], dtype=bool)
    cliente['es_trabajador'] = np.array(unities[2], dtype=bool)
    cliente['facultad'] = np.array(fac,dtype=str)
    cliente['instituto'] = np.array(ins, dtype=str)
    cliente['unidad'] = np.array(uni, dtype=str)
    cliente['email'] = np.array(emails, dtype=str)
    
    # Verificamos que todos los curps sean distintos
    assert len(set(list(cliente['curp']))) == n_client
    
    # Escribimos CSV
    
    cliente.to_csv(c.filepath_client, index=False)

In [3]:
cliente

,curp,nombre,a_paterno,a_materno,calle_num,municipio,cp,colonia,estado,tel_casa,tel_celular,email,fotografia,facultad,instituto,unidad,es_alumno,es_academico,es_trabajador
0,HEAJ810722MCYHJD86,JOSE LUIS,HEREDIA,ABAD,Angel R Cavada Mz 57 L 42,Álvaro Obregón,15000,Jalalpa Grande,Cuidad de México,5628688184,5574390078,joseheredia81@cepe.unam.mx,proyecto/data/photos/photo_HEAJ810722MCYHJD86.png,No aplica,No aplica,CEPE,False,False,True
1,COMM040711MZEISW91,ALBERTO,CONDE,MONTELLANO,Pacifico 274 Estac 14,Coyoacán,43500,Rosedal,Cuidad de México,5585615010,5673662599,albertoconde04@arquitectura.unam.mx,proyecto/data/photos/photo_COMM040711MZEISW91.png,Facultad de Arquitectura,No aplica,No aplica,True,False,False
2,GOCF010401MVJEEW81,JOSE,GOMEZ,CANSECO,"Avd. PACIFICO, 350 Dpto. E-001",Coyoacán,43300,EL ROSEDAL,Cuidad de México,5518394543,5606718628,josegomez01@arquitectura.unam.mx,proyecto/data/photos/photo_GOCF010401MVJEEW81.png,Facultad de Arquitectura,No aplica,No aplica,True,False,False
3,GALJ020515MJWEKY44,JORGE,GARCIA,LA,Cartzo 119 Local a,Cuauhtémoc,64000,Sta Maria la Rivera,Cuidad de México,5642525739,5530778735,jorgela02@psicologia.unam.mx,proyecto/data/photos/photo_GALJ020515MJWEKY44.png,Facultad de Psicología,Instituto de Biolgía,No aplica,True,True,False
4,ZUSJ031007MSAFKP77,MANUEL,ZUÑIGA,SANTOS,Camelia M-7 L-9,Álvaro Obregón,17400,San Clemente,Cuidad de México,5671754571,5543187179,manuelsantos03@ciencias.unam.mx,proyecto/data/photos/photo_ZUSJ031007MSAFKP77.jpg,Facultad de Ciencias,No aplica,No aplica,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,GACG010405FQRITY56,MARGARITA,GARCIA,CONTRERAS,Camino a Sta. Cruz Num. 53,Iztapalapa,97900,Lomas de San Lorenzo,Cuidad de México,5574091918,5603807924,margaritacontreras01@mvz.unam.mx,proyecto/data/photos/photo_GACG010405FQRITY56.jpg,Facultad de Medicina Veterinaria y Zootecnia,No aplica,No aplica,True,False,False
9996,AGDG050820FOJXQP16,GABRIELA,AGUILERA,DEOLARTE,"Prv. EMPERADOR, MZ-1 LT-38",Magdalena Contreras,10369,TIERRA UNIDA,Cuidad de México,5581272589,5523467723,gabrieladeolarte05@quimica.unam.mx,proyecto/data/photos/photo_AGDG050820FOJXQP16.jpg,Facultad de Química,No aplica,No aplica,True,False,False
9997,GUCG040522FNAYCW29,MARIA DE LOUR,GUTIERREZ,CARDENAS,Darwin 48,Miguel Hidalgo,11590,Anzures,Cuidad de México,5665840154,5604006928,mariagutierrez04@derecho.unam.mx,proyecto/data/photos/photo_GUCG040522FNAYCW29.png,Facultad de Derecho,No aplica,No aplica,True,False,False
9998,MOAG020916FHVEKI69,LAURA ESTELA,MOJICA,ALVAREZ,"2DO AND 2 DE ABRIL, MZ-5 LT-3-A",Magdalena Contreras,10900,PUEBLO SAN NICOLAS TOTOLAPAN,Cuidad de México,5508290382,5513457431,lauraalvarez02@ciencias.unam.mx,proyecto/data/photos/photo_MOAG020916FHVEKI69.jpg,Facultad de Ciencias,No aplica,No aplica,True,False,False


## Entidad Socio

In [7]:
try:
    socio = pd.read_csv(c.filepath_partner)

except:
    print('Creando entidad Socio...')
    
    n_socio = c.cardinality['partner']
    
    # nombres
    data_names_masc = pd.read_csv(c.path_masc_names, encoding='latin-1')
    data_names_fem = pd.read_csv(c.path_fem_names, encoding='latin-1')
    data_surnames = pd.read_csv(c.path_surnames, encoding='latin-1')

    names_list = u.get_names(data_names_masc, 
                             data_names_fem, 
                             data_surnames, 
                             n_socio, 
                             2)
    
    # Names
    names = names_list[0]
    surnames_1 = names_list[1]
    surnames_2 = names_list[2]
    names_masc = names_list[3]
    names_fem = names_list[4]
    _, n_masc, n_fem = names_list[5]
    
    # Curps
    curp_masc = list(u.get_curps(names_masc,
                             surnames_1[0: n_masc],
                             surnames_2[0: n_masc],
                             sex='M', socio=True))
    curp_fem = list(u.get_curps(names_fem,
                            surnames_1[n_masc:],
                            surnames_2[n_masc:],
                            sex='F', socio=True))
    curps = curp_masc + curp_fem
    rfcs = np.array(u.get_rfc(curps), dtype=str)
    
    # Direcciones
    cdmx_data = pd.read_csv(c.filepath_raw + '/all_cdmx_data.csv')
    np.random.seed(2)
    to_select = np.random.choice(np.arange(0, cdmx_data.index[-1]), size=n_socio, replace=True)
    cdmx_socio = cdmx_data.iloc[to_select]
    cdmx_socio.astype({'cp': 'float32'})
    
    ingreso, egreso = u.get_in_out(n_socio)
    
    # Dueños y choferes
    choferes, duenios = u.get_due_cho(n_socio)
    
    # Creamos y rellenamos
    socio = pd.DataFrame(r.dict_to_df_socio)
    
    socio['id_socio'] = np.array([f's-{id}' for id in range(1, n_socio+1)], dtype=str)
    socio['nombre'] = np.array(names, dtype=str)
    socio['a_paterno'] = np.array(surnames_1, dtype=str)
    socio['a_materno'] = np.array(surnames_2, dtype=str)
    socio['rfc'] = rfcs
    socio['num_licencia'] = np.array(u.get_lic(choferes), dtype=str)
    socio['calle_num'] = np.array(cdmx_socio['calle_num'], dtype=str)
    socio['municipio'] = np.array(cdmx_socio['alcaldia'], dtype=str)
    socio['cp'] = np.array(cdmx_socio['cp'], dtype=int)
    socio['colonia'] = np.array(cdmx_socio['colonia'], dtype=str)
    socio['estado'] = str(c.address)
    socio['celular'] = np.array(u.get_tels(n_socio), dtype=int)
    socio['email'] = np.array(u.get_simple_mail(names), dtype=str)
    socio['fotografia'] = np.array(u.get_photos(rfcs), dtype=str)
    socio['ingreso'] = ingreso
    socio['egreso'] = egreso
    socio['es_chofer'] = np.array(choferes, dtype=bool)
    socio['es_duenio'] = np.array(duenios, dtype=bool)
    
    # Escribimos CSV
    socio.to_csv(c.filepath_partner, index=False)
    

Creando entidad Socio...


In [5]:
socio

,id_socio,nombre,a_paterno,a_materno,rfc,num_licencia,calle_num,colonia,cp,municipio,estado,celular,email,fotografia,ingreso,egreso,es_chofer,es_duenio
0,s-1,LUIS ENRIQUE,FLORES,SOSA,FLSJ860406MKU7,S53745449,Zapote Secc.2 Edif.centauri 139 202-a,Romulo Sanchez Mireles,14070,Tlalpan,Cuidad de México,5522353121,luis777@hotmail.com,proyecto/data/photos/photo_FLSJ860406MKU7.jpg,2021-10-15,No aplica,True,True
1,s-2,HUMBERTO,HERNANDEZ,CAMACHO,HECM800814MLC1,T05186522,Froa#o 34 B 202,Valle Gomez,15210,Venustiano Carranza,Cuidad de México,5565590412,humberto643@gmail.com,proyecto/data/photos/photo_HECM800814MLC1.jpg,2011-01-18,No aplica,True,False
2,s-3,RODOLFO,GUERRERO,GUTIERREZ,GUGF830709MXE5,No aplica,Rio Usumacinta 33 Mza. 10 Lt. 14,Ampl. Lomas de San Bernabe,10300,Magdalena Contreras,Cuidad de México,5625717050,rodolfo709@gmail.com,proyecto/data/photos/photo_GUGF830709MXE5.jpg,2015-05-19,No aplica,False,True
3,s-4,RUBEN,VILLANUEVA,MEZA,VIMJ770114MGU4,B28290500,Halacho Lt 13 a Mz 806,San Nicolas Totolapan,14100,Tlalpan,Cuidad de México,5520305069,ruben946@gmail.com,proyecto/data/photos/photo_VIMJ770114MGU4.jpg,2018-02-10,2018-07-30,True,False
4,s-5,RAFAEL,SAN,GASPAR,SAGJ791012MGY1,H82224922,Edif 6 Ent D Depto 203,Unidad Lindavista Vallejo,77200,Gustavo A. Madero,Cuidad de México,5501206771,rafael610@hotmail.com,proyecto/data/photos/photo_SAGJ791012MGY1.jpg,2016-05-14,No aplica,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,s-196,LUZ DEL CARMEN,HERNANDEZ,DOMINGUEZ,HEDG830701FVW7,No aplica,"Eje 1 PONIENTE PROL GUERRERO, 340 Edif. ALLEND...",NONOALCO TLATELOLCO,69000,Cuauhtémoc,Cuidad de México,5577105502,luz615@gmail.com,proyecto/data/photos/photo_HEDG830701FVW7.png,2014-04-11,No aplica,False,True
196,s-197,TANIA KARINA,LOPEZ,ZAVALA,LOZD830111FOI3,Q04984366,Esperanza S/n,Santo Domingo,21400,Azcapotzalco,Cuidad de México,5618593478,tania377@hotmail.com,proyecto/data/photos/photo_LOZD830111FOI3.png,2010-05-19,No aplica,True,False
197,s-198,RICARDA,RIVERA,RAMIREZ,RIRI740315FMF5,N77666450,Nezahualcoyotl Z 5 Mz 132 Lt 26,Sta Ma Aztahuacan,95000,Iztapalapa,Cuidad de México,5661081736,ricarda436@yahoo.com,proyecto/data/photos/photo_RIRI740315FMF5.png,2019-01-12,No aplica,True,False
198,s-199,MARIA ALEJANDRA,FLORES,AGUILAR,FLAA810326FVM5,P54783204,Aguascalientes Z 1 Mz 69 Lt 4,Chalma de Guadalupe,72100,Gustavo A. Madero,Cuidad de México,5618138031,maria529@gmail.com,proyecto/data/photos/photo_FLAA810326FVM5.png,2015-02-28,No aplica,True,False


In [6]:
sum(socio['egreso'] == 'No aplica')

173